# CIT 5450 Final Project
Joanne Crean, Juan Goleniowski, Joseph Ricci

Before running, be sure to create a new Kaggle API token and save it as 'kaggle.json' in your default Google Drive location.

In [1]:
# Installs
!pip install -q kaggle

In [2]:
# Imports
import pandas as pd
from google.colab import drive
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Setup
drive.mount('/content/drive')
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
# Download Datasets
# Spotify Top 200 and Viral 50 Charts for 2017 through 2021 Dataset
!!kaggle datasets download -d dhruvildave/spotify-charts

# Spotify Audio Features Datasets
!!kaggle datasets download -d rodolfofigueroa/spotify-12m-songs
!!kaggle datasets download -d muhmores/spotify-top-100-songs-of-20152019
!!kaggle datasets download -d sashankpillai/spotify-top-200-charts-20202021
!!kaggle datasets download -d maharshipandya/-spotify-tracks-dataset
!!kaggle datasets download -d vatsalmavani/spotify-dataset
!!kaggle datasets download -d nandhakumarss/spotify-song-tracks
!!kaggle datasets download -d elemento/music-albums-popularity-prediction

# TikTok Popular Songs Dataset
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2019
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2020
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2021

['Downloading tiktok-popular-songs-2021.zip to /content',
 '',
 '  0% 0.00/11.0k [00:00<?, ?B/s]',
 '',
 '100% 11.0k/11.0k [00:00<00:00, 7.41MB/s]']

In [5]:
# Unzip Datasets
!unzip /content/spotify-charts.zip
!unzip /content/spotify-12m-songs.zip
!unzip /content/spotify-top-100-songs-of-20152019.zip
!unzip /content/spotify-top-200-charts-20202021.zip
!unzip /content/-spotify-tracks-dataset.zip
!unzip /content/spotify-dataset.zip
!unzip /content/spotify-song-tracks.zip
!unzip /content/music-albums-popularity-prediction.zip
!unzip /content/tiktok-popular-songs-2019
!unzip /content/tiktok-popular-songs-2020
!unzip /content/tiktok-popular-songs-2021

Archive:  /content/spotify-charts.zip
  inflating: charts.csv              
Archive:  /content/spotify-12m-songs.zip
  inflating: tracks_features.csv     
Archive:  /content/spotify-top-100-songs-of-20152019.zip
  inflating: Spotify 2010 - 2019 Top 100 Songs.xlsx  
  inflating: Spotify 2010 - 2019 Top 100.csv  
Archive:  /content/spotify-top-200-charts-20202021.zip
  inflating: spotify_dataset.csv     
Archive:  /content/-spotify-tracks-dataset.zip
  inflating: dataset.csv             
Archive:  /content/spotify-dataset.zip
  inflating: data/data.csv           
  inflating: data/data_by_artist.csv  
  inflating: data/data_by_genres.csv  
  inflating: data/data_by_year.csv   
  inflating: data/data_w_genres.csv  
Archive:  /content/spotify-song-tracks.zip
  inflating: SpotifyFeatures.csv     
Archive:  /content/music-albums-popularity-prediction.zip
  inflating: sample_solution.csv     
  inflating: test.csv                
  inflating: train.csv               
Archive:  /content/tiktok

In [6]:
# Clean up directory to save space
!rm sample_data/*
!rm -d sample_data
!rm ./*.zip
!rm sample_solution.csv
!rm ./*.xlsx

rm: cannot remove 'sample_data/*': No such file or directory
rm: cannot remove 'sample_data': No such file or directory


In [7]:
# Read the csv files and save them to pandas dataframes
df_charts = pd.read_csv('charts.csv')
df_song_features_1 = pd.read_csv('tracks_features.csv')
df_song_features_2 = pd.read_csv('Spotify 2010 - 2019 Top 100.csv')
df_song_features_3 = pd.read_csv('spotify_dataset.csv')
df_song_features_4 = pd.read_csv('dataset.csv')
df_song_features_5 = pd.read_csv('data/data.csv')
df_song_features_6 = pd.read_csv('SpotifyFeatures.csv')
df_song_features_7 = pd.read_csv('train.csv')
df_song_features_10 = pd.read_csv('test.csv')
df_tiktok_19 = pd.read_csv('TikTok_songs_2019.csv')
df_tiktok_20 = pd.read_csv('TikTok_songs_2020.csv')
df_tiktok_21 = pd.read_csv('TikTok_songs_2021.csv')

In [8]:
# Clean up remaining files
!rm data/*
!rm -d data
!rm ./*.csv

# Spotify Top 200 and Viral 50 Charts EDA

In [9]:
# Create a separate column for Year
df_charts['Year']=df_charts['date'].str.slice(stop=4)

# Cast 'Year' as Int
df_charts['Year'] = df_charts['Year'].astype(int)

# Show datatypes 
df_charts.dtypes

#remove ID, trend and streams columns
df_charts.drop(['url', 'trend', 'streams'], axis=1, inplace=True)

In [10]:
# Number of record per year
df_charts.groupby(by='Year').count()

,title,rank,date,artist,region,chart
Year,,,,,,
2017,4401145,4401145,4401145,4401145,4401145,4401145
2018,4863016,4863016,4863016,4863016,4863016,4863016
2019,5412658,5412669,5412669,5412669,5412669,5412669
2020,5793148,5793148,5793148,5793130,5793148,5793148
2021,5703536,5703536,5703536,5703536,5703536,5703536


In [11]:
# Remore records with year 2017 and 2018
df_charts=df_charts[df_charts['Year'] > 2018]

In [12]:
df_charts.groupby(by='Year').count()

,title,rank,date,artist,region,chart
Year,,,,,,
2019,5412658,5412669,5412669,5412669,5412669,5412669
2020,5793148,5793148,5793148,5793130,5793148,5793148
2021,5703536,5703536,5703536,5703536,5703536,5703536


# Spotify Audio Feature Dataset EDA

Since the Spotify Top 200 and Viral 50 charts dataset does not come with audio features the songs, there is no guarantee that we can find datasets that include audio features for all of the songs in the charts. One relevant question to explore is therefore

1. Can we find audio feature data for a suitably "big" fraction of the songs in the Charts dataset?

Additionally, since we'll be combining multiple audio features datasets, there is no guarantee that those datasets share the same features, so

2. What fields do the datasets share?

In [13]:
# First, since we want to find song features for as many of the songs in 'df_charts' as possible, let's start by
# projecting df_charts by song title and artist name, which will uniquely identify each song, and then dropping duplicates
df_charts_songs_artists = df_charts[['title', 'artist']].drop_duplicates()

# Next, let's join the features datasets on song title and artist name and see what percentage of songs in charts_df
# each song features dataset can provide features for

# Clean extranneous characters from artist columns if necessary
df_song_features_1['artists'] = df_song_features_1['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_5['artists'] = df_song_features_5['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_7['artists'] = df_song_features_7['artists'].str.replace(', ', '')
df_song_features_10['artists'] = df_song_features_10['artists'].str.replace(', ', '')

# Project only the necessary columns from each feature dataset and also rename to maintain consistency
features_1_projected = df_song_features_1[['name', 'artists']].rename(columns={'name': 'title', 'artists': 'artist'})
features_2_projected = df_song_features_2[['title', 'artist']]
features_3_projected = df_song_features_3[['Song Name', 'Artist']].rename(columns={'Song Name': 'title', 'Artist': 'artist'})
features_4_projected = df_song_features_4[['track_name', 'artists']].rename(columns={'track_name': 'title', 'artists': 'artist'})
features_5_projected = df_song_features_5[['name', 'artists']].rename(columns={'name': 'title', 'artists': 'artist'})
features_6_projected = df_song_features_6[['track_name', 'artist_name']].rename(columns={'track_name': 'title', 'artist_name': 'artist'})
features_7_projected = df_song_features_7[['t_name0', 'artists']].rename(columns={'t_name0': 'title', 'artists': 'artist'})
features_8_projected = df_song_features_7[['t_name1', 'artists']].rename(columns={'t_name1': 'title', 'artists': 'artist'})
features_9_projected = df_song_features_7[['t_name2', 'artists']].rename(columns={'t_name2': 'title', 'artists': 'artist'})
features_10_projected = df_song_features_10[['t_name0', 'artists']].rename(columns={'t_name0': 'title', 'artists': 'artist'})
features_11_projected = df_song_features_10[['t_name1', 'artists']].rename(columns={'t_name1': 'title', 'artists': 'artist'})
features_12_projected = df_song_features_10[['t_name2', 'artists']].rename(columns={'t_name2': 'title', 'artists': 'artist'})

# Join on song title and artist name
merge_1 = pd.merge(df_charts_songs_artists, features_1_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_2 = pd.merge(df_charts_songs_artists, features_2_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_3 = pd.merge(df_charts_songs_artists, features_3_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_4 = pd.merge(df_charts_songs_artists, features_4_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_5 = pd.merge(df_charts_songs_artists, features_5_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_6 = pd.merge(df_charts_songs_artists, features_6_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_7 = pd.merge(df_charts_songs_artists, features_7_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_8 = pd.merge(df_charts_songs_artists, features_8_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_9 = pd.merge(df_charts_songs_artists, features_9_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_10 = pd.merge(df_charts_songs_artists, features_10_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_11 = pd.merge(df_charts_songs_artists, features_11_projected, on=['title', 'artist'], how='inner').drop_duplicates()
merge_12 = pd.merge(df_charts_songs_artists, features_12_projected, on=['title', 'artist'], how='inner').drop_duplicates()

# Calculate the percentage of songs each features dataset was able to provide song features for
df_combined_features = pd.concat([merge_1, merge_2, merge_3, merge_4, merge_5, merge_6, merge_7, merge_8, merge_9, merge_10, merge_11, merge_12], ignore_index=True).drop_duplicates()
match_percentage_combined = 100 * df_combined_features.shape[0] / df_charts_songs_artists.shape[0]
print("We are able to provide audio features for " + str(round(match_percentage_combined,2)) + "% of songs in df_charts,"\
      " for a total of " + str(df_combined_features.shape[0]) + " out of " + str(df_charts_songs_artists.shape[0]) + " songs")

We are able to provide audio features for 16.48% of songs in df_charts, for a total of 18950 out of 115016 songs


# TikTok Popular Songs Dataset EDA

# Genius Lyrics Dataset Creation and EDA